In [3]:
import pandas as pd
import sqlite3
import os

In [4]:
!ls 

DJANGO	   Untitled.ipynb	      Write_to_djangoDBv3_mulitPlot.ipynb
myweb	   Write_to_djangoDB.ipynb
README.md  Write_to_djangoDBv2.ipynb


In [5]:
djangoDB_path = './myweb/db.sqlite3'

# res 為　db.sqlite3　裡面有的table

In [6]:
conn = sqlite3.connect(djangoDB_path)
c = conn.cursor()

# Create table
c.execute('''select name from sqlite_master where type='table' order by name;''')
res = c.fetchall()
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

In [7]:
import pandas as pd
from pymongo import MongoClient
from pymongo import errors
import numpy as np

In [8]:
def connectMongoDB(port):
    #uri = "mongodb://192.168.1.44" 
    client = MongoClient(port = port, serverSelectionTimeoutMS=1)
    _ = client.server_info()
    db = client["kafka"]
    return db
db = connectMongoDB(27017)

# 從monogoDB 取出　並　寫入　db.sqlite3

In [9]:
conn = sqlite3.connect(djangoDB_path)
c = conn.cursor()


test_fund = pd.DataFrame(list( db["test_fund"].find({})))
test_fund = test_fund.drop("_id",1)
test_fund = test_fund[["date", "price", "blueP", "redP"]]
#test_fund = test_fund.fillna(-1)
# Insert a row of data
test_fund.to_sql("mainsite_historyprice", if_exists="replace", con=conn, index_label="id")

# Save (commit) the changes
conn.commit()
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

# 　re_test_fund　=>  查看被寫入db.sqlite3的資料

In [11]:
conn = sqlite3.connect(djangoDB_path)

re_test_fund = pd.read_sql_query("SELECT * FROM mainsite_historyprice", conn)

# Save (commit) the changes
conn.commit()
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

# 將資料寫到要呈現的JS檔

In [73]:
demo_chart_path = "myweb/myweb/static/mysite/js/demo"
viewData = re_test_fund.tail(60)

#viewData = viewData.fillna(round(viewData.price.min()*0.7))

In [74]:
# {'dict', 'list', 'series', 'split', 'records', 'index'}


141.96

In [100]:
categories = "\n'categories':[{\n'category': %s}]"%(
    viewData[["date"]].rename(columns={"date":"label"}).to_dict("records")
)
PriceData = "\n'seriesname': 'Pirce',\n 'anchorBgColor':'#849e93',\n 'data':%s"%(
    viewData[["price"]].rename(columns={"price":"value"}).astype(str).to_dict("records")
)
RedPriceData = "\n'seriesname': 'RedSign',\n 'anchorBgColor':'#d30e49',\n 'data':%s"%(
    viewData[["redP"]].rename(columns={"redP":"value"}).astype(str).to_dict("records")
)
BluePriceData = "\n'seriesname': 'BlueSign',\n 'anchorBgColor':'#1e44ed',\n 'data':%s"%(
    viewData[["blueP"]].rename(columns={"blueP":"value"}).astype(str).to_dict("records")
)

dataset = "'dataset':[{%s\n},{%s\n},{%s\n}]"%(PriceData, RedPriceData, BluePriceData)
chart = '\n"chart": {\n"caption": "Social Media Platforms Popularity",\n"subCaption": "2012-2016",\n"theme": "hulk-light",\n"showhovereffect": "1",\n"xAxisName": "Years",\n"showValues": "%d",\n"drawCrossLine": "1",\n"crossLineAlpha": "100",\n"crossLineColor": "#cc3300"\n   }'%(viewData.price.min())

dataSource = "\ndataSource:{\n  %s,\n %s ,\n %s }"%(chart, categories, dataset)

FusionCharts = "{\n%s,\n %s,\n %s,\n %s,\n %s,\n %s}"%("type: 'msline'",
                                           "renderAt: 'chart-container'" ,
                                           "width : '100%'",
                                           "height: '400'",
                                           "dataFormat: 'json'",
                                           dataSource
                                          )
toJSFile = "FusionCharts.ready(\nfunction() {\nvar popularityChart = new FusionCharts(\n%s\n).render();\n});"%FusionCharts

In [101]:
with open(os.path.join(demo_chart_path, "multi-series2.js"), "w") as writer:
    writer.write(toJSFile)

In [89]:
RedPriceData

"\n'seriesname': 'RedSign',\n 'anchorBgColor':'#d30e49',\n 'data':[{'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'

In [88]:
BluePriceData

"\n'seriesname': 'BlueSign',\n 'anchorBgColor':'#1e44ed',\n 'data':[{'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan'}, {'value': 'nan

# Price

In [75]:
POINT_COLOR = "'rgba(53, 149, 50, 1)'"## 點的顏色


DATA_DATASETS_LABEL = "'Sessions'"
DATA_DATASETS_LINETENSION = 0.3
DATA_DATASETS_BACKGROUNDCOLOR = "'rgba(2,117,216,0.2)'"
DATA_DATASETS_BORDERCOLOR = POINT_COLOR
DATA_DATASETS_POINTRADIUS = 2
DATA_DATASETS_POINTBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTBORDERCOLOR = "'rgba(255,255,255,0.8)'"
DATA_DATASETS_POINTHOVERRADIUS = 2
DATA_DATASETS_POINTHOVERBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTHITRADIUS = 2
DATA_DATASETS_POINTBORDERWIDTH = 1
DATA_DATASETS_DATA = viewData.price.values.tolist()

data_datasets_label = "label: %s"%(DATA_DATASETS_LABEL)
data_datasets_lineTension = "lineTension: %d"%(DATA_DATASETS_LINETENSION)
data_datasets_backgroundColor = "backgroundColor: %s"%(DATA_DATASETS_BACKGROUNDCOLOR)
data_datasets_borderColor = "borderColor: %s"%(DATA_DATASETS_BORDERCOLOR)
data_datasets_pointRadius = "pointRadius: %d"%(DATA_DATASETS_POINTRADIUS)
data_datasets_pointBackgroundColor = "pointBackgroundColor: %s"%(DATA_DATASETS_POINTBACKGROUNDCOLOR)
data_datasets_pointBorderColor = "pointBorderColor: %s"%(DATA_DATASETS_POINTBORDERCOLOR)
data_datasets_pointHoverRadius = "pointHoverRadius: %d"%(DATA_DATASETS_POINTHOVERRADIUS)
data_datasets_pointHoverBackgroundColor = "pointHoverBackgroundColor: %s"%(DATA_DATASETS_POINTHOVERBACKGROUNDCOLOR)
data_datasets_pointHitRadius = "pointHitRadius: %d"%(DATA_DATASETS_POINTHITRADIUS)
data_datasets_pointBorderWidth = "pointBorderWidth: %d"%(DATA_DATASETS_POINTBORDERWIDTH)
data_datasets_data = "data: %s"%(DATA_DATASETS_DATA)

data_datasets = "datasets:[{%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s}]"%(
    data_datasets_label,
    data_datasets_lineTension,
    data_datasets_backgroundColor,
    data_datasets_borderColor,
    data_datasets_pointRadius,
    data_datasets_pointBackgroundColor,
    data_datasets_pointBorderColor,
    data_datasets_pointHoverRadius,
    data_datasets_pointHoverBackgroundColor,
    data_datasets_pointHitRadius,
    data_datasets_pointBorderWidth,
    data_datasets_data,)

DATA_LABELS = viewData.date.values.tolist()
data_labels = "labels: %s"%(DATA_LABELS)
_data = "data:{%s,%s}"%(data_labels, data_datasets)

OPTIONS_SCALES_XAXES_TIME = "'date'"
OPTIONS_SCALES_XAXES_GRIDLINES = "false"
OPTIONS_SCALES_XAXES_TICKS = 5
OPTIONS_SCALES_YAXES_TICKS_MIN = round(viewData.price.min()*0.7)
OPTIONS_SCALES_YAXES_TICKS_MAX = round(viewData.price.max()*1.3)
OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT = 5
OPTIONS_SCALES_YAXES_GRIDLINES = "'rgba(0, 0, 0, .125)'"
OPTIONS_LENGEND = "false"

options_scales_xAxes_ticks = "ticks: {maxTicksLimit: %d}"%(OPTIONS_SCALES_XAXES_TICKS)        
options_scales_xAxes_gridLines = "gridLines: {display: %s}"%(OPTIONS_SCALES_XAXES_GRIDLINES)
options_scales_xAxes_time = "time: {unit: %s}"%(OPTIONS_SCALES_XAXES_TIME)
options_scales_xAxes = "xAxes: [{%s,%s,%s}]"%(
    options_scales_xAxes_time, 
    options_scales_xAxes_gridLines,
    options_scales_xAxes_ticks
)

options_scales_yAxes_gridLines = "gridLines: {color: %s}"%(OPTIONS_SCALES_YAXES_GRIDLINES)
options_scales_yAxes_ticks = "ticks: {min:%f, max:%f, maxTicksLimit:%d}"%(OPTIONS_SCALES_YAXES_TICKS_MIN,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAX,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT
                                                                         )     
options_scales_yAxes = "yAxes: [{%s,%s}]"%(
    options_scales_yAxes_ticks, 
    options_scales_yAxes_gridLines,
)
options_scales = "scales:{%s,%s}"%(options_scales_xAxes, options_scales_yAxes)

options_legend = "legend:{ display :%s}"%(OPTIONS_LENGEND)
options = "options:{%s, %s}"%(options_scales, options_legend)


TYPE = 'line'
_type = "type: '%s'"%(TYPE)

postJson = "{%s, %s, %s}"%(_type, _data, options)
PriceLineChart = "var PriceLineChart = new Chart(Pctx, %s);"%(postJson)

# Red Sign

In [76]:
redP_viewData  = viewData.dropna(subset=["redP"])

if redP_viewData.empty:
    redP_viewData_price_list = []
    redP_viewData_date_list  = []
    OPTIONS_SCALES_YAXES_TICKS_MIN = round(viewData.price.min()*0.7)
    OPTIONS_SCALES_YAXES_TICKS_MAX = round(viewData.price.max()*1.3)
else:
    redP_viewData_price_list = redP_viewData.redP.values.tolist()
    redP_viewData_date_list  = redP_viewData.date.values.tolist()
    OPTIONS_SCALES_YAXES_TICKS_MIN = round((redP_viewData.redP.min())*0.7)
    OPTIONS_SCALES_YAXES_TICKS_MAX = round((redP_viewData.redP.max())*1.3)

POINT_COLOR = "'rgba(236, 24, 27, 1)'"  ## 點的顏色


DATA_DATASETS_LABEL = "'Sessions'"
DATA_DATASETS_LINETENSION = 0
DATA_DATASETS_BACKGROUNDCOLOR = "'rgba(2,117,216,0.2)'"
DATA_DATASETS_BORDERCOLOR = POINT_COLOR
DATA_DATASETS_POINTRADIUS = 5
DATA_DATASETS_POINTBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTBORDERCOLOR = "'rgba(255, 0, 0, 0.7)'"
DATA_DATASETS_POINTHOVERRADIUS = 5
DATA_DATASETS_POINTHOVERBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTHITRADIUS = 2
DATA_DATASETS_POINTBORDERWIDTH = 1
DATA_DATASETS_DATA = redP_viewData_price_list

data_datasets_label = "label: %s"%(DATA_DATASETS_LABEL)
data_datasets_lineTension = "lineTension: %d"%(DATA_DATASETS_LINETENSION)
data_datasets_backgroundColor = "backgroundColor: %s"%(DATA_DATASETS_BACKGROUNDCOLOR)
data_datasets_borderColor = "borderColor: %s"%(DATA_DATASETS_BORDERCOLOR)
data_datasets_pointRadius = "pointRadius: %d"%(DATA_DATASETS_POINTRADIUS)
data_datasets_pointBackgroundColor = "pointBackgroundColor: %s"%(DATA_DATASETS_POINTBACKGROUNDCOLOR)
data_datasets_pointBorderColor = "pointBorderColor: %s"%(DATA_DATASETS_POINTBORDERCOLOR)
data_datasets_pointHoverRadius = "pointHoverRadius: %d"%(DATA_DATASETS_POINTHOVERRADIUS)
data_datasets_pointHitRadius = "pointHitRadius: %d"%(DATA_DATASETS_POINTHITRADIUS)
data_datasets_pointBorderWidth = "pointBorderWidth: %d"%(DATA_DATASETS_POINTBORDERWIDTH)
data_datasets_data = "data: %s"%(DATA_DATASETS_DATA)

data_datasets = "datasets:[{%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s}]"%(
    data_datasets_label,
    data_datasets_lineTension,
    data_datasets_backgroundColor,
    data_datasets_borderColor,
    data_datasets_pointRadius,
    data_datasets_pointBackgroundColor,
    data_datasets_pointBorderColor,
    data_datasets_pointHoverRadius,
    data_datasets_pointHoverBackgroundColor,
    data_datasets_pointHitRadius,
    data_datasets_pointBorderWidth,
    data_datasets_data,)

DATA_LABELS = redP_viewData_date_list
data_labels = "labels: %s"%(DATA_LABELS)
_data = "data:{%s,%s}"%(data_labels, data_datasets)

OPTIONS_SCALES_XAXES_TIME = "'date'"
OPTIONS_SCALES_XAXES_GRIDLINES = "false"
OPTIONS_SCALES_XAXES_TICKS = 5
OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT = 5
OPTIONS_SCALES_YAXES_GRIDLINES = "'rgba(0, 0, 0, .125)'"
OPTIONS_LENGEND = "false"

options_scales_xAxes_ticks = "ticks: {maxTicksLimit: %d}"%(OPTIONS_SCALES_XAXES_TICKS)        
options_scales_xAxes_gridLines = "gridLines: {display: %s}"%(OPTIONS_SCALES_XAXES_GRIDLINES)
options_scales_xAxes_time = "time: {unit: %s}"%(OPTIONS_SCALES_XAXES_TIME)
options_scales_xAxes = "xAxes: [{%s,%s,%s}]"%(
    options_scales_xAxes_time, 
    options_scales_xAxes_gridLines,
    options_scales_xAxes_ticks
)

options_scales_yAxes_gridLines = "gridLines: {color: %s}"%(OPTIONS_SCALES_YAXES_GRIDLINES)
options_scales_yAxes_ticks = "ticks: {min:%f, max:%f, maxTicksLimit:%d}"%(OPTIONS_SCALES_YAXES_TICKS_MIN,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAX,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT
                                                                         )     
options_scales_yAxes = "yAxes: [{%s,%s}]"%(
    options_scales_yAxes_ticks, 
    options_scales_yAxes_gridLines,
)
options_scales = "scales:{%s,%s}"%(options_scales_xAxes, options_scales_yAxes)

options_legend = "legend:{ display :%s}"%(OPTIONS_LENGEND)
options = "options:{%s, %s}"%(options_scales, options_legend)


TYPE = 'line'
_type = "type: '%s'"%(TYPE)

postJson = "{%s, %s, %s}"%(_type, _data, options)
RedPLineChart = "var RedPLineChart = new Chart(Rctx, %s);"%(postJson)

# Blue Sign

In [77]:
blueP_viewData = viewData.dropna(subset=["blueP"])

if blueP_viewData.empty:
    blueP_viewData_price_list = []
    blueP_viewData_date_list  = []
    OPTIONS_SCALES_YAXES_TICKS_MIN = round(viewData.price.min()*0.7)
    OPTIONS_SCALES_YAXES_TICKS_MAX = round(viewData.price.max()*1.3)
else:
    blueP_viewData_price_list = blueP_viewData.blueP.values.tolist()
    blueP_viewData_date_list  = blueP_viewData.date.values.tolist()
    OPTIONS_SCALES_YAXES_TICKS_MIN = round((blueP_viewData.blueP.min())*0.7)
    OPTIONS_SCALES_YAXES_TICKS_MAX = round((blueP_viewData.blueP.max())*1.3)

POINT_COLOR = "'rgba(99, 145, 242, 1)'"  ## 點的顏色

DATA_DATASETS_LABEL = "'Sessions'"
DATA_DATASETS_LINETENSION = 0
DATA_DATASETS_BACKGROUNDCOLOR = "'rgba(2,117,216,0.2)'"
DATA_DATASETS_BORDERCOLOR = POINT_COLOR
DATA_DATASETS_POINTRADIUS = 5
DATA_DATASETS_POINTBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTBORDERCOLOR = "'rgba(255, 0, 0, 0.7)'"
DATA_DATASETS_POINTHOVERRADIUS = 5
DATA_DATASETS_POINTHOVERBACKGROUNDCOLOR = POINT_COLOR
DATA_DATASETS_POINTHITRADIUS = 2
DATA_DATASETS_POINTBORDERWIDTH = 1
DATA_DATASETS_DATA = blueP_viewData_price_list

data_datasets_label = "label: %s"%(DATA_DATASETS_LABEL)
data_datasets_lineTension = "lineTension: %d"%(DATA_DATASETS_LINETENSION)
data_datasets_backgroundColor = "backgroundColor: %s"%(DATA_DATASETS_BACKGROUNDCOLOR)
data_datasets_borderColor = "borderColor: %s"%(DATA_DATASETS_BORDERCOLOR)
data_datasets_pointRadius = "pointRadius: %d"%(DATA_DATASETS_POINTRADIUS)
data_datasets_pointBackgroundColor = "pointBackgroundColor: %s"%(DATA_DATASETS_POINTBACKGROUNDCOLOR)
data_datasets_pointBorderColor = "pointBorderColor: %s"%(DATA_DATASETS_POINTBORDERCOLOR)
data_datasets_pointHoverRadius = "pointHoverRadius: %d"%(DATA_DATASETS_POINTHOVERRADIUS)
data_datasets_pointHitRadius = "pointHitRadius: %d"%(DATA_DATASETS_POINTHITRADIUS)
data_datasets_pointBorderWidth = "pointBorderWidth: %d"%(DATA_DATASETS_POINTBORDERWIDTH)
data_datasets_data = "data: %s"%(DATA_DATASETS_DATA)

data_datasets = "datasets:[{%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s}]"%(
    data_datasets_label,
    data_datasets_lineTension,
    data_datasets_backgroundColor,
    data_datasets_borderColor,
    data_datasets_pointRadius,
    data_datasets_pointBackgroundColor,
    data_datasets_pointBorderColor,
    data_datasets_pointHoverRadius,
    data_datasets_pointHoverBackgroundColor,
    data_datasets_pointHitRadius,
    data_datasets_pointBorderWidth,
    data_datasets_data,)

DATA_LABELS = blueP_viewData_date_list
data_labels = "labels: %s"%(DATA_LABELS)
_data = "data:{%s,%s}"%(data_labels, data_datasets)

OPTIONS_SCALES_XAXES_TIME = "'date'"
OPTIONS_SCALES_XAXES_GRIDLINES = "false"
OPTIONS_SCALES_XAXES_TICKS = 5
OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT = 5
OPTIONS_SCALES_YAXES_GRIDLINES = "'rgba(0, 0, 0, .125)'"
OPTIONS_LENGEND = "false"

options_scales_xAxes_ticks = "ticks: {maxTicksLimit: %d}"%(OPTIONS_SCALES_XAXES_TICKS)        
options_scales_xAxes_gridLines = "gridLines: {display: %s}"%(OPTIONS_SCALES_XAXES_GRIDLINES)
options_scales_xAxes_time = "time: {unit: %s}"%(OPTIONS_SCALES_XAXES_TIME)
options_scales_xAxes = "xAxes: [{%s,%s,%s}]"%(
    options_scales_xAxes_time, 
    options_scales_xAxes_gridLines,
    options_scales_xAxes_ticks
)

options_scales_yAxes_gridLines = "gridLines: {color: %s}"%(OPTIONS_SCALES_YAXES_GRIDLINES)
options_scales_yAxes_ticks = "ticks: {min:%f, max:%f, maxTicksLimit:%d}"%(OPTIONS_SCALES_YAXES_TICKS_MIN,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAX,
                                                                          OPTIONS_SCALES_YAXES_TICKS_MAXTICKSLIMIT
                                                                         )     
options_scales_yAxes = "yAxes: [{%s,%s}]"%(
    options_scales_yAxes_ticks, 
    options_scales_yAxes_gridLines,
)
options_scales = "scales:{%s,%s}"%(options_scales_xAxes, options_scales_yAxes)

options_legend = "legend:{ display :%s}"%(OPTIONS_LENGEND)
options = "options:{%s, %s}"%(options_scales, options_legend)


TYPE = 'line'
_type = "type: '%s'"%(TYPE)

postJson = "{%s, %s, %s}"%(_type, _data, options)
BluePLineChart = "var BluePLineChart = new Chart(Bctx, %s);"%(postJson)

In [78]:
postJSON = "%s\n%s\n%s\n%s\n%s\n%s\n%s\n%s\n"%(
    "Chart.defaults.global.defaultFontFamily = '%s';"%('-apple-system,system-ui,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,sans-serif'),
    "Chart.defaults.global.defaultFontColor = '#292b2c';",
    "var Pctx = document.getElementById('myAreaChart');",
    "var Rctx = document.getElementById('myRed');",
    "var Bctx = document.getElementById('myBlue');",
    PriceLineChart,
    RedPLineChart,
    BluePLineChart
)

In [79]:
with open(os.path.join(demo_chart_path, "my-chart-area-demo2.js"), "w") as writer:
    writer.write(postJSON)